In [1]:
import math
import numpy as np

In [2]:
def multiplicative_order(a, n):
    """
    Вычисляет порядок числа a по модулю n
    """
    k = 1; flag = True # начнем перебор с единицы

    while flag:
        if (a ** k - 1) % n == 0: # если порядок найден
            flag = False # "опускаем" флаг и выходим из цикла
        else: # иначе
            k += 1 # увеличиваем порядок на единицу

    return k

In [3]:
print(multiplicative_order(10, 107))
print(multiplicative_order(2, 15))

53
4


In [4]:
def euclidean_algorithm_extended(a, b):
    """
    Находит d = НОД(a, b), а также такие целые числа x и y, что ax + by = d, с помощью расширенного алгоритма Евклида
    """
    (a, b) = (int(a), int(b))

    reversed = True if abs(b) > abs(a) else False # флаг
    (a, b) = (b, a) if reversed else (a, b) # меняем местами a и b, если нужно

    (r, x, y) = ([a, b], [1, 0], [0, 1]) # шаг 1

    while r[1] != 0: 
        (r[0], r[1], q) = (r[1], r[0] % r[1], r[0] // r[1])

        if r[1] != 0: # если остаток ещё не нулевой..
            (x[0], x[1]) = (x[1], x[0] - q * x[1])
            (y[0], y[1]) = (y[1], y[0] - q * y[1])

    (d, x_r, y_r) = (r[0], x[1], y[1])

    if reversed: # если a и b были в неправильном порядке
        (x_r, y_r) = (y_r, x_r) # меняем найденные коэффициенты местами

    return (d, x_r, y_r)

def solve_congruence(c, d, p):
    """
    Решает сравнение вида k_1 * x + b_1 = k_2 * x + b_2 (mod p), где
    c = (k_1, b_1), d = (k_2, b_2)
    """
    (k_1, b_1) = c; (k_2, b_2) = d # получаем коэффициенты

    k = k_1 - k_2; b = b_2 - b_1 # kx = b (mod p)
    (gcd, k_inverse, _) = euclidean_algorithm_extended(k, p) # k * k_inverse = gcd (mod p)

    if gcd == 1: # если k и p - взаимно простые..
        return (b * k_inverse) % p
    else: # иначе
        k = int(k / gcd); b = int(b / gcd) # делим сравнение на gcd
        (_, k_inverse, _) = euclidean_algorithm_extended(k, int(p / gcd))

        return (b * k_inverse) % p

In [5]:
def pollard_rho_dlog(a, b, p, def0 = True, to_print = False):
    """
    Решает сравнение a^x = b (mod p) ро-методом Полларда;
    def0 = True, если нужно использовать начальные значения u и v по умолчанию, и False, если их нужно определить случайно;
    to_print = True, если нужно вывести на экран ход алгоритма
    """
    r = multiplicative_order(a, p) # порядок числа а
    half_p = math.floor(p / 2) # p / 2

    # отображение f
    f = "({a} * x % {p}) if x < {half} else ({b} * x % {p})".format(a = a, p = p, half = half_p, b = b)

    # начальные значения u и v
    (u, v) = (2, 2) if def0 else (np.random.randint(1, half_p), np.random.randint(1, half_p))

    if not def0 and to_print:
        print("(u, v) = ({}, {})".format(u, v))

    c = ((a ** u) * (b ** v)) % p #  
    d = c                         #
                                  # шаг 1
    (k_c, l_c) = (u, v)           # 
    (k_d, l_d) = (u, v)           #

    if to_print:
        print("{:^10} | {:^10} | {:^10} | {:^10}".format("c", "log_c", "d", "log_d"))
        print("{:^10} | {:^10} | {:^10} | {:^10}".format("----------", "----------", "----------", "----------"))
        print("{:^10} | {:^3} + {:^3}x | {:^10} | {:^3} + {:^3}x".format(c, l_c, k_c, d, l_d, k_d))

    while True:
        # вычисляем f(c) 
        # и log_a f(c)
        x = c             #
        if x < half_p:    #
            l_c += 1      #
        else:             #
            k_c += 1      #
        c = eval(f)       #
                          #
        # вычисляем f(c)  #
        # и log_a f(c)    #
        x = d             #  шаг 2
        if x < half_p:    #
            l_d += 1      #
        else:             #
            k_d += 1      #
        x = eval(f)       #
        if x < half_p:    #
            l_d += 1      #
        else:             #
            k_d += 1      #
        d = eval(f)       #

        if to_print:
            print("{:^10} | {:^3} + {:^3}x | {:^10} | {:^3} + {:^3}x".format(c, l_c, k_c, d, l_d, k_d))

        # шаг 3
        if c == d:
            # приравниваем логарифмы и решаем сравнение
            result = solve_congruence((k_c, l_c), (k_d, l_d), r)

            if (a ** result - b) % p == 0: # проверка
                return result
            else:
                return 0

In [6]:
pollard_rho_dlog(10, 64, 107, True, True)

    c      |   log_c    |     d      |   log_d   
---------- | ---------- | ---------- | ----------
    4      |  2  +  2 x |     4      |  2  +  2 x
    40     |  3  +  2 x |     79     |  4  +  2 x
    79     |  4  +  2 x |     56     |  5  +  3 x
    27     |  4  +  3 x |     75     |  5  +  5 x
    56     |  5  +  3 x |     3      |  5  +  7 x
    53     |  5  +  4 x |     86     |  7  +  7 x
    75     |  5  +  5 x |     42     |  8  +  8 x
    92     |  5  +  6 x |     23     |  9  +  9 x
    3      |  5  +  7 x |     53     | 11  +  9 x
    30     |  6  +  7 x |     92     | 11  + 11 x
    86     |  7  +  7 x |     30     | 12  + 12 x
    47     |  7  +  8 x |     47     | 13  + 13 x


20

In [10]:
pollard_rho_dlog(5, 3, 23, False, True)

(u, v) = (7, 3)
    c      |   log_c    |     d      |   log_d   
---------- | ---------- | ---------- | ----------
    22     |  3  +  7 x |     22     |  3  +  7 x
    20     |  3  +  8 x |     14     |  3  +  9 x
    14     |  3  +  9 x |     11     |  3  + 11 x
    19     |  3  + 10 x |     4      |  4  + 12 x
    11     |  3  + 11 x |     14     |  5  + 13 x
    10     |  3  + 12 x |     11     |  5  + 15 x
    4      |  4  + 12 x |     4      |  6  + 16 x


16

In [11]:
pollard_rho_dlog(29, 479, 797, True, False)

3